In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import pudl
from pudl.etl import defs

In [ ]:
logger = pudl.logging_helpers.get_logger(__name__)

In [ ]:
def plot_filled_rolled_plant(frc):
    plant_id =frc[frc.fuel_cost_per_mmbtu_source.isin(["eiaapi", "rolling_avg"])].plant_id_eia.sample().values[0]
    plant = frc[frc.plant_id_eia == plant_id]
    for fuel in plant.fuel_type_code_pudl.unique():
        plant_fuel = plant[plant.fuel_type_code_pudl == fuel]
        # plt.plot(plant_fuel.report_date, plant_fuel.fuel_cost_per_mmbtu, label=f"Fully Imputed {fuel}")
        for source in plant_fuel.fuel_cost_per_mmbtu_source.unique():
            plant_fuel_source = plant_fuel[plant_fuel.fuel_cost_per_mmbtu_source == source]
            plt.scatter(plant_fuel_source.report_date, plant_fuel_source.fuel_cost_per_mmbtu, label=f"{source} {fuel}")
    plt.ylabel("Fuel cost $/mmBTU")
    plt.xlabel("Report Date")
    plt.title(f"Results of FRC Table for {plant_fuel.plant_name_eia.values[0]}")
    plt.legend(loc="upper left")
    plt.show()

In [ ]:
frc = defs.load_asset_value("out_eia923__fuel_receipts_costs")

In [ ]:
## percentages of records from the various fuel cost sources
(frc.fuel_cost_per_mmbtu_source.value_counts(dropna=False)/ len(frc)).round(2)

In [ ]:
# re-run this cell to see a new sample plant
plot_filled_rolled_plant(frc)

In [ ]:
(
    frc.assign(report_year=lambda x: x.report_date.dt.year)
    .groupby(["report_year","fuel_cost_per_mmbtu_source"], observed=True)
    [["fuel_cost_per_mmbtu"]].sum().unstack(["fuel_cost_per_mmbtu_source"])
    .plot(title="Annual sum of Fuel Cost by Source",ylabel="fuel_cost_per_mmbtu")
)
plt.show()

In [ ]:
(
    frc.assign(report_year=lambda x: x.report_date.dt.year)
    .groupby(["fuel_cost_per_mmbtu_source","energy_source_code"], observed=True)
    [["fuel_cost_per_mmbtu"]].sum().unstack(["fuel_cost_per_mmbtu_source"])
    .plot(title="Total Fuel Cost by Source",ylabel="fuel_cost_per_mmbtu")
)
plt.show()

In [ ]:
(
    frc.assign(report_year=lambda x: x.report_date.dt.year)
    .groupby(["fuel_cost_per_mmbtu_source","energy_source_code"], observed=True)
    [["total_fuel_cost"]].sum().unstack(["fuel_cost_per_mmbtu_source"])
    .plot(title="Total Fuel Cost by Fuel Cost Source and Energy Source Code",ylabel="total_fuel_cost")
)
plt.show()

In [ ]:
def log_total_mmbtu_by_source(frc, source):
    perc = frc[(frc.fuel_cost_per_mmbtu_source == source) &frc.fuel_cost_per_mmbtu.notnull()].fuel_consumed_mmbtu.sum()/ frc.fuel_consumed_mmbtu.sum()
    logger.info(f"{source}:   {perc:.1%}")
logger.info("Total mmBTU w/ Fuel cost data:")
log_total_mmbtu_by_source(frc, "original")
log_total_mmbtu_by_source(frc, "eiaapi")
log_total_mmbtu_by_source(frc, "rolling_avg")
log_total_mmbtu_by_source(frc, pd.NA)

In [ ]:
plt.figure(figsize=(8,6))
range=(0,10)
bins=40
alpha=.6
for source in ["original", "eiaapi", "rolling_avg"]:
    frc_source = frc[frc.fuel_cost_per_mmbtu_source == source]
    plt.hist(frc_source.fuel_cost_per_mmbtu,range=range,bins=bins,
             label=source,alpha=alpha,
             weights=frc_source.fuel_consumed_mmbtu
            );
plt.ylabel("mmBTU")
plt.xlabel("$/mmBTU")
plt.title("FRC Table by Fuel Cost Source")
plt.legend()
plt.show()